# Using neural networks

In this tutorial, we will use a classic neural network named Multilayer Perceptron, or MLP.

This is a very generic network that consists of the composition of several single perceptrons, as shown in the image below. The input data flows forwardly throughout the different layers, until it reaches the output nodes. The learning is performed using the same Gradient Descent technique that we have seen during the course! (With a bit more complex formula, due to the composition of gradients).


<img src="img/mlp.png" width="600">

First, let's import the necessary libraries.

In [29]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

The aim of this guide is to build a classification model to detect diabetes. For this, we will be using [Kaggle's diabetes dataset](https://www.kaggle.com/datasets/mathchi/diabetes-data-set). 

Ps: Don't you know what Kaggle is? Ask your instructor!

Load the dataset, contained in the `data/` folder, and print show the first 5 records. You can use function `read_csv` for this: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [30]:
diabetes = pd.read_csv("data/diabetes.csv")

How many observations and variables does the dataset contain?

In [31]:
diabetes.shape
#768 observations, 9 variables

(768, 9)

The different variables for this dataset are described as follows:
* Pregnancies - Number of times pregnant.
* Glucose - Plasma glucose concentration.
* BloodPressure - Diastolic blood pressure (mm Hg).
* SkinThickness - Skinfold thickness (mm).
* Insulin - Hour serum insulin (mu U/ml).
* BMI – Basal metabolic rate (weight in kg/height in m).
* DiabetesPedigreeFunction - Diabetes pedigree function.
* Age - Age in years.
* Outcome - “1” represents the presence of diabetes while “0” represents the absence of it. This is the variable we want to create a predictor on.

Show some basic statistics for the dataset variables. You can use pandas' `describe()` for this purpose: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html

In [32]:
diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Looking at the summary for the 'Outcome' variable, we observe that the mean value is 0.35, which means that around 35 percent of the observations in the dataset have diabetes. Therefore, the baseline accuracy is 65 percent and our neural network model should definitely beat this baseline benchmark.

Create 2 lists. One containing one element, the target variable name, and the other containing the other 8 predictor variables. We will use these lists to benefit from the pandas' slicing operators

In [33]:
target_column = ['Outcome']
print(target_column)
predictors = diabetes.columns
print(predictors)
print(type(predictors))
predictors = predictors.drop(['Outcome'])
print(predictors)

['Outcome']
Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')
<class 'pandas.core.indexes.base.Index'>
Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')


Normalize the predictive variables to have a maximum value of 1 and a minimum value of 0. For this, you can do your own implementation, or use sklearn's `MinMaxScaler` function: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

Use again pandas' `describe` function to verify the correctness of your approach

In [34]:
diabetes[predictors] = (diabetes[predictors]-diabetes[predictors].min())/(diabetes[predictors].max()-diabetes[predictors].min())
y = diabetes[target_column]
X = diabetes[predictors]

print(type(y))
y.describe()
X.describe()



<class 'pandas.core.frame.DataFrame'>


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,0.226180,0.607510,0.566438,0.207439,0.094326,0.476790,0.168179,0.204015
std,0.198210,0.160666,0.158654,0.161134,0.136222,0.117499,0.141473,0.196004
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.058824,0.497487,0.508197,0.000000,0.000000,0.406855,0.070773,0.050000
50%,0.176471,0.587940,0.590164,0.232323,0.036052,0.476900,0.125747,0.133333
75%,0.352941,0.704774,0.655738,0.323232,0.150414,0.545455,0.234095,0.333333
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [35]:
print(not(1) or not(0) in y.index) # je vérifie si la colonne 'Output' est bien constituée seulement de 0 et de 1

False


Slice the dataset into using the previously created indices, to craft your model's input and target

Use sklearn's `train_test_split` to split your dataset into a train and a test cohort. The test size should comprise the 30% of the total size. Use a _random\_state_ of 40: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [36]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=40,stratify=y)

Print the size of both your training and testing to verify that the split was done properly.

In [37]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(537, 8)
(537, 1)
(231, 8)
(231, 1)


Time to model our Multilayer Perceptron! For this, you can use sklearn's `MLPClassifier` function: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
You can use the reference documentation to set 3 hidden layers, each with a composed of 8 neurons, and the maximum number of iterations to 500.

To train the model you can use the `fit()` function, as seen during the course.

If you encounter a sklearn warning about lack of convergence, you can increase a bit the argument _max\_iter_. But beware that you could run into an overfitting situation!

In [38]:
print(y_train.sum()/y_train.shape[0])
print(y_test.sum()/y_test.shape[0])

Outcome    0.348231
dtype: float64
Outcome    0.350649
dtype: float64


Use the model's `predict()` function to obtain the predictions for the training set.

In [39]:
mlp = MLPClassifier(hidden_layer_sizes = (8,8,8), activation = "relu", random_state=42, max_iter=500).fit(X_train, np.ravel(y_train))
predict_train = mlp.predict(X_train)
predict_train


array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,

Once the predictions are generated, we can evaluate the performance of the model. Being a classification algorithm, we would like to check the accuracy metrics. However, since the dataset is not completely balanced, the precision, recall, and f1 metrics are also very interesting to us.

Let's use sklearn's `confusion_matrix` function to obtain the confusion matrix from the training predictions: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

In [40]:
confusion_matrix(y_train,predict_train)
# TN , FP
# FN , TP

array([[314,  36],
       [ 70, 117]], dtype=int64)

Sklearn also provides a function to conveniently verify the performance of our model. Use this function, `classification_report`, to see our performance: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

In [41]:
# Use a print statement for a better visualization

print(classification_report(y_train, predict_train))

              precision    recall  f1-score   support

           0       0.82      0.90      0.86       350
           1       0.76      0.63      0.69       187

    accuracy                           0.80       537
   macro avg       0.79      0.76      0.77       537
weighted avg       0.80      0.80      0.80       537



While results look promising, lets recall that all these are done on data already seen, as this is the data we have train with. Repeat the same process with the test predictions, and verify that the performance is still good.

In [42]:
predict_test = mlp.predict(X_test)
confusion_matrix(y_test,predict_test)
# TN , FP
# FN , TP
print(classification_report(y_test, predict_test))

              precision    recall  f1-score   support

           0       0.80      0.85      0.82       150
           1       0.68      0.62      0.65        81

    accuracy                           0.77       231
   macro avg       0.74      0.73      0.74       231
weighted avg       0.76      0.77      0.76       231



We have also improved the baseline performance with unseen data. That is very good news!

The model can be further improved by doing cross-validation, feature engineering, or changing the arguments in the neural network estimator. Try to iterate and beat these results!

You can also compare your work with the notebooks provided in the [Kaggle's dataset code section](https://www.kaggle.com/datasets/mathchi/diabetes-data-set/code).